In [1]:
from __future__ import division

import CalculateCoElutionScores as CS
import GoldStandard as GS
import utils as utils
import sys
import copy
import os
import argparse
import pickle
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Using Theano backend.


utils.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/kuan-hao/miniconda3/envs/EPIC/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/kuan-hao/miniconda3/envs/EPIC/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/kuan-hao/miniconda3/envs/EPIC/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kuan-hao/miniconda3/envs/EPIC/lib/python2.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/kuan-hao/miniconda3/envs/EPIC/lib/python2.7/site-packages/ipykernel/kernelapp.py", lin

In [2]:
feature_selection = '000000001'
input_dir = '/home/kuan-hao/EPPC/input/real_data/elution_profiles_test/'
taxid = ''
ppi = ''
cluster = '/home/kuan-hao/EPPC/input/real_data/gold_standard.tsv'
output_dir_ = '/home/kuan-hao/EPPC/output/TEST/'
output_prefix = 'TEST'
classifier = 'CNN'
num_cores = 6
mode = 'EXP'
fun_anno_source = 'STRING'
co_elution_cutoff = 0.5
classifier_cutoff = 0.5
elution_max_count = 1
frac_count = 2
precalcualted_score_file = "NONE"
K_D_TRAIN = 'd'
FOLD_NUM = 5
TRAIN_TEST_RATIO = 0.3
POS_NEG_RATIO = 1
NUM_EP = 2
NUM_FRC = 60

In [3]:
if feature_selection == "00000000":
    print "Select at least one feature"
    sys.exit()
elif feature_selection == "000000000":
    this_scores = utils.get_fs_comb(feature_selection)
    print "\t".join([fs.name for fs in this_scores])
elif feature_selection == "000000001":
    this_scores = utils.get_fs_comb(feature_selection)
    print "\t".join([fs.name for fs in this_scores])
else:
    this_scores = utils.get_fs_comb(feature_selection)
    print "\t".join([fs.name for fs in this_scores])

('this_scores: ', [<CalculateCoElutionScores.Raw_eps instance at 0x7f65b8201998>])
Raw_eps


In [4]:
foundprots, elution_datas = utils.load_data(input_dir, this_scores, fc=frac_count, mfc=elution_max_count)

finished processing /home/kuan-hao/EPPC/input/real_data/elution_profiles_test/intensity_high_500.tsv
 removed 23 (0.05, total: 499, after filtering: 476) proteins found in less than 2 fraction


In [5]:
##########################################
## "Generate reference data set"
##########################################
gs = ""
if ((taxid != "" and  ppi != "") or (cluster != "" and  ppi != "" )):
    print "Refernce from cluster and PPI are nor compatiple. Please supply ppi or complex reference, not both!"
    sys.exit()

if taxid == "" and  ppi == "" and cluster == "":
    print "Please supply a reference by setting taxid, cluster, or ppi tag"
    sys.exit()

gs_clusters = []
if (taxid != "" and cluster == "" and ppi == ""):
    print "Loading clusters from GO, CORUM, and Intact"
    gs_clusters.extend(utils.get_reference_from_net(taxid))

if cluster != "":
    print "Loading complexes from file"
    print("&& args.cluster: ", cluster)
    print("&& args.mode: ", mode)
    if mode == "FA":
        gs_clusters.append(GS.FileClusters(cluster, "all"))
    else:
        gs_clusters.append(GS.FileClusters(cluster, foundprots))


if ppi != "":
    print "Reading PPI file from %s" % reference
    gs = Goldstandard_from_PPI_File(ppi, foundprots)

print gs_clusters
if 	len(gs_clusters)>0:
    gs = utils.create_goldstandard(gs_clusters, taxid, foundprots, pos_neg_ratio = POS_NEG_RATIO)


output_dir = output_dir_ + os.sep + output_prefix

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

refFH = open(output_dir + ".ref_complexes.txt", "w")
for comp in gs.complexes.complexes:
        print >> refFH, "%s\t%s" % (",".join(comp), ",".join(gs.complexes.complexes[comp]))
refFH.close()
##########################################
## End of "Generate reference data set"
##########################################

Loading complexes from file
('&& args.cluster: ', '/home/kuan-hao/EPPC/input/real_data/gold_standard.tsv')
('&& args.mode: ', 'EXP')
Average size of predicted complexes is: 4.84695652174
[<GoldStandard.FileClusters instance at 0x7f65b8206320>]
Total number of complexes 190 in Goldstandard
Number of complexes after ortholog mapping 190 complexes in Goldstandard
After removing not indetified proteins 190 number of complexes in Goldstandard
After size filtering 30 number of complexes in Goldstandard
After mergning 28 number of complexes in Goldstandard


In [6]:
##########################################
## "Creating classifier"
##########################################
classifier_select = classifier
clf = CS.CLF_Wrapper(num_cores, classifier_select, num_ep=NUM_EP, num_frc=NUM_FRC, pos_neg_ratio = POS_NEG_RATIO)
##########################################
## End of "Creating classifier"
##########################################

Using CNN


In [7]:
##########################################
## Calculating coelutionScores
##########################################
scoreCalc = CS.CalculateCoElutionScores(this_scores, elution_datas, output_dir + ".scores.txt", num_cores=num_cores, cutoff= co_elution_cutoff)
if precalcualted_score_file == "NONE":
    scoreCalc.calculate_coelutionDatas(gs)
else:
    scoreCalc.readTable(precalcualted_score_file, gs)

print("scoreCalc.scores.shape: ", scoreCalc.scores.shape)

## Use autoencoder to extract features

##########################################
## End of "Calculating coelutionScores"
##########################################

Filtering: intensity_high_500.tsv
Before filtering 113050 PPIs
filtering
After filtering 81609 PPIs
Num of PPIs across all data sets after filtering 81609
('num_rows: ', 5671)
('len(self.features): ', 1)
('num_frc: ', 27)
done calcualting co-elution scores
('** self.scores: ', array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.66223253, 0.12831917, 0.04597004, ..., 0.        ,
         0.        , 0.        ]],

       [[0.02134712, 0.01442621, 0.04481638, ..., 0.00409582,
         0.00422245, 0.00711002],
        [0.59756116, 0.28238223, 0.12005661, ..., 0.        ,
         0.        , 0.        ]],

       [[0.31466232, 0.42925407, 0.18827869, ..., 0.        ,
         0.        , 0.        ],
        [0.01416298, 0.0016486 , 0.00148367, ..., 0.        ,
         0.        , 0.        ]],

       ...,

       [[0.        , 0.        , 0.03770384, ..., 0.        ,
         0.        , 0.        ],
        [0.17714545, 0.        

In [8]:
##########################################
## "Balancing Positive & Negative" PPIs
##########################################
functionalData = ""
gs.positive = set(gs.positive & set(scoreCalc.ppiToIndex.keys()))
gs.negative = set(gs.negative & set(scoreCalc.ppiToIndex.keys()))
gs.all_positive = gs.positive
gs.all_negative = gs.negative
# gs.rebalance()

print("len(gs.positive): ", len(gs.positive))
print("len(gs.negative): ", len(gs.negative))

print("len(gs.all_positive): ", len(gs.all_positive))
print("len(gs.all_negative): ", len(gs.all_negative))
###############################################
## End of "Balancing Positive & Negative" PPIs
###############################################

('len(gs.positive): ', 244)
('len(gs.negative): ', 4768)
('len(gs.all_positive): ', 244)
('len(gs.all_negative): ', 4768)


In [9]:
_, data, targets, _all, data_all, targets_all = scoreCalc.toSklearnData(gs)
print("data.shape", data.shape)
print("targets.shape", targets.shape)
# print("_all.shape", _all.shape)
print("data_all.shape", data_all.shape)
print("targets_all.shape", targets_all.shape)

('&& self.scores.shape[0]: ', 5012)
('&& pos_count: ', 244)
('&& neg_count: ', 4768)
('&& self.scores.shape[0]: ', 5012)
('&& pos_count_all: ', 244)
('&& neg_count_all: ', 4768)
('&& unsure_count_all: ', 0)
('data.shape', (5012, 2, 27))
('targets.shape', (5012,))
('data_all.shape', (5012, 2, 27))
('targets_all.shape', (5012,))


In [10]:
data = np.array(data)
num_samples, num_scores, num_frc = data.shape
print("data: ", data.shape)
data = data.reshape((num_samples, num_scores*num_frc))
print("data.shape: ", data.shape)
print(data[0])
print("targets.shape: ", targets.shape)
# print(targets)
from sklearn.model_selection import train_test_split, StratifiedKFold
x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size=0.3, random_state=1)
print("     x_train: ", x_train.shape)
print("     x_test: ", x_test.shape)
print("     y_train: ", y_train.shape)
print("     y_test: ", y_test.shape)

('data: ', (5012, 2, 27))
('data.shape: ', (5012, 54))
[0.         0.         0.         0.         0.         0.48754491
 0.51245509 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.66223253 0.12831917 0.04597004
 0.0683785  0.04115505 0.05033422 0.0036105  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
('targets.shape: ', (5012,))
('     x_train: ', (3508, 54))
('     x_test: ', (1504, 54))
('     y_train: ', (3508,))
('     y_test: ', (1504,))


In [11]:
import keras
from keras import layers

## Autoencoder compression

In [12]:
# input_img = keras.Input(shape=(54,))
# encoded = layers.Dense(27, activation='relu')(input_img)
# encoded = layers.Dense(14, activation='relu')(encoded)
# # encoded = layers.Dense(7, activation='relu')(encoded)
# encoded = layers.Dense(5, activation='relu')(encoded)


# decoded = layers.Dense(14, activation='relu')(encoded)
# # decoded = layers.Dense(30, activation='relu')(decoded)
# decoded = layers.Dense(27, activation='relu')(decoded)
# decoded = layers.Dense(54, activation='sigmoid')(decoded)

# This is the size of our encoded representations
encoding_dim = 5  # 5 floats -> compression of factor 10.8, assuming the input is 54 floats

# This is our input image
input_img = keras.Input(shape=(54,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(27, activation='relu')(input_img)
encoded = layers.Dense(14, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)

# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(14, activation='relu')(encoded)
# decoded = layers.Dense(30, activation='relu')(decoded)
decoded = layers.Dense(27, activation='relu')(decoded)
decoded = layers.Dense(54, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [13]:
# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

In [14]:
# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [15]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [16]:
history = autoencoder.fit(x_train, x_train,
                epochs=300,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 3508 samples, validate on 1504 samples
Epoch 1/300
3508/3508 [==============================] - 0s 5us/step - loss: 0.6909 - val_loss: 0.6860
Epoch 2/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.6817 - val_loss: 0.6764
Epoch 3/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.6714 - val_loss: 0.6651
Epoch 4/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.6589 - val_loss: 0.6511
Epoch 5/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.6432 - val_loss: 0.6336
Epoch 6/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.6238 - val_loss: 0.6121
Epoch 7/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.6002 - val_loss: 0.5861
Epoch 8/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.5720 - val_loss: 0.5557
Epoch 9/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.5399 - val_loss: 0.5219
Epoch 10/300
3508/3508 [=

3508/3508 [==============================] - 0s 4us/step - loss: 0.1338 - val_loss: 0.1346
Epoch 80/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1337 - val_loss: 0.1345
Epoch 81/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1336 - val_loss: 0.1344
Epoch 82/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1336 - val_loss: 0.1343
Epoch 83/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1335 - val_loss: 0.1342
Epoch 84/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1334 - val_loss: 0.1341
Epoch 85/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1333 - val_loss: 0.1340
Epoch 86/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1332 - val_loss: 0.1339
Epoch 87/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1331 - val_loss: 0.1338
Epoch 88/300
3508/3508 [==============================] - 0s 4us/step - loss:

3508/3508 [==============================] - 0s 6us/step - loss: 0.1244 - val_loss: 0.1252
Epoch 158/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1242 - val_loss: 0.1250
Epoch 159/300
3508/3508 [==============================] - 0s 5us/step - loss: 0.1241 - val_loss: 0.1249
Epoch 160/300
3508/3508 [==============================] - 0s 5us/step - loss: 0.1240 - val_loss: 0.1248
Epoch 161/300
3508/3508 [==============================] - 0s 5us/step - loss: 0.1239 - val_loss: 0.1247
Epoch 162/300
3508/3508 [==============================] - 0s 5us/step - loss: 0.1237 - val_loss: 0.1246
Epoch 163/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1236 - val_loss: 0.1244
Epoch 164/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1235 - val_loss: 0.1243
Epoch 165/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1234 - val_loss: 0.1242
Epoch 166/300
3508/3508 [==============================] - 0s 4us/ste

Epoch 235/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1168 - val_loss: 0.1180
Epoch 236/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1167 - val_loss: 0.1179
Epoch 237/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1166 - val_loss: 0.1178
Epoch 238/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1166 - val_loss: 0.1177
Epoch 239/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1165 - val_loss: 0.1176
Epoch 240/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1164 - val_loss: 0.1176
Epoch 241/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1163 - val_loss: 0.1175
Epoch 242/300
3508/3508 [==============================] - 0s 5us/step - loss: 0.1162 - val_loss: 0.1174
Epoch 243/300
3508/3508 [==============================] - 0s 4us/step - loss: 0.1161 - val_loss: 0.1173
Epoch 244/300
3508/3508 [==============================

In [26]:
import matplotlib.pyplot as plt


In [28]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(os.path.join("/home/kuan-hao/Desktop/loss.png"), dpi=400)
plt.close()

In [23]:
# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)
print("len(encoded_imgs): ", len(encoded_imgs))
print("len(encoded_imgs[0]): ", len(encoded_imgs[0]))
print("len(decoded_imgs[0]): ", len(decoded_imgs[0]))

('len(encoded_imgs): ', 1504)
('len(encoded_imgs[0]): ', 5)
('len(decoded_imgs[0]): ', 54)


## Transform input into encoded format

In [25]:
data_encoded = encoder.predict(data)

In [28]:
print("len(data_encoded): ", len(data_encoded))
print("len(targets): ", len(targets))

('len(data_encoded): ', 5012)
('len(targets): ', 5012)


In [29]:
# this_targets, preds, probs, precision, recall, fmeasure, auc_pr, auc_roc, curve_pr, curve_roc = clf.cv_eval(data, targets, outDir, k_d_training, folds, train_test_ratio, num_ep, num_frc)


from sklearn.model_selection import train_test_split, StratifiedKFold
x_train, x_test, y_train, y_test = train_test_split(data_encoded, targets, test_size=0.3, random_state=1)
print("     x_train: ", x_train.shape)
print("     x_test: ", x_test.shape)
print("     y_train: ", y_train.shape)
print("     y_test: ", y_test.shape)

('     x_train: ', (3508, 5))
('     x_test: ', (1504, 5))
('     y_train: ', (3508,))
('     y_test: ', (1504,))


In [30]:
print "Processing data..."
self.clf = CNN_raw_ef_model(num_ep, num_frc)
self.clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
self.fit(x_train, y_train)
probs = self.predict_proba(x_test)
preds = self.predict(x_test)
this_targets = y_test

array([[0.95898783, 1.7111433 , 1.6578858 , 3.301536  , 2.1571996 ],
       [2.0985763 , 1.7226908 , 1.7012873 , 0.        , 2.3623118 ],
       [1.1075162 , 1.0938008 , 1.613379  , 1.5816376 , 1.5809512 ],
       ...,
       [0.5186173 , 2.2352295 , 1.7778525 , 2.1587675 , 1.0290148 ],
       [1.4964948 , 1.4587287 , 0.92274106, 0.670108  , 2.1766384 ],
       [1.5855032 , 1.2713962 , 1.3904507 , 2.7344291 , 1.7089255 ]],
      dtype=float32)

In [9]:
# ##########################################
# ## This part is "EVALUATION"
# ##########################################
# print "Start benchmarking"

# if mode == "EXP":
#     utils.cv_bench_clf(scoreCalc, clf, gs, output_dir, format="pdf", verbose=True, k_d_training = K_D_TRAIN, folds = FOLD_NUM, train_test_ratio = TRAIN_TEST_RATIO, num_ep = NUM_EP, num_frc = NUM_FRC)

# if mode == "COMB":
#     tmp_sc = copy.deepcopy(scoreCalc)
#     tmp_sc.add_fun_anno(functionalData)
#     utils.cv_bench_clf(tmp_sc, clf, gs, output_dir, format="pdf", verbose=True, k_d_training = K_D_TRAIN, folds = FOLD_NUM, train_test_ratio = TRAIN_TEST_RATIO, num_ep = NUM_EP, num_frc = NUM_FRC)

# if mode == "FA":
#     utils.cv_bench_clf(functionalData, clf, gs, output_dir, format="pdf", verbose=True, k_d_training = K_D_TRAIN, folds = FOLD_NUM, train_test_ratio = TRAIN_TEST_RATIO, num_ep = NUM_EP, num_frc = NUM_FRC)
# ##########################################
# ## End of "EVALUATION"
# ##########################################

Start benchmarking
('&& self.scores.shape[0]: ', 5012)
('&& pos_count: ', 244)
('&& neg_count: ', 244)
('&& self.scores.shape[0]: ', 5012)
('&& pos_count_all: ', 244)
('&& neg_count_all: ', 4768)
('&& unsure_count_all: ', 0)
('data: ', (488, 2, 27))
('data.shape: ', (488, 2, 27))
('targets.shape: ', (488,))
('data_all: ', (5012, 2, 27))
('data_all.shape: ', (5012, 2, 27))
('targets_all.shape: ', (5012,))
('data: ', (488, 2, 27))
('data.shape: ', (488, 2, 27, 1))
[[[0.        ]
  [0.03388044]
  [0.05531865]
  [0.05226366]
  [0.04394045]
  [0.09155187]
  [0.02190318]
  [0.04774131]
  [0.03144744]
  [0.07235891]
  [0.06894582]
  [0.31839759]
  [0.05387115]
  [0.07679509]
  [0.01711927]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.00577184]
  [0.00869332]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.05082859]
  [0.0

341/341 [==============================] - 0s 123us/step - loss: 0.2064 - acc: 0.9091
Epoch 141/200
341/341 [==============================] - 0s 124us/step - loss: 0.1701 - acc: 0.9267
Epoch 142/200
341/341 [==============================] - 0s 137us/step - loss: 0.2118 - acc: 0.9003
Epoch 143/200
341/341 [==============================] - 0s 135us/step - loss: 0.1606 - acc: 0.9413
Epoch 144/200
341/341 [==============================] - 0s 135us/step - loss: 0.1763 - acc: 0.9296
Epoch 145/200
341/341 [==============================] - 0s 133us/step - loss: 0.1672 - acc: 0.9150
Epoch 146/200
341/341 [==============================] - 0s 126us/step - loss: 0.1698 - acc: 0.9267
Epoch 147/200
341/341 [==============================] - 0s 125us/step - loss: 0.1734 - acc: 0.9179
Epoch 148/200
341/341 [==============================] - 0s 125us/step - loss: 0.1566 - acc: 0.9296
Epoch 149/200
341/341 [==============================] - 0s 125us/step - loss: 0.1802 - acc: 0.9091
Epoch 150/200


In [10]:
##############################################
## This part is "MODEL TRAINING & PREDICTION"
##############################################
network = utils.make_predictions(scoreCalc, mode, clf, gs, output_dir, fun_anno=functionalData, verbose = False, k_d_training = K_D_TRAIN, folds = FOLD_NUM, train_test_ratio = TRAIN_TEST_RATIO)

# Predict protein interaction
outFH = open("%s.pred.txt" % (output_dir), "w")

final_network = []
for PPI in network:
    items = PPI.split("\t")
    if float(items[2]) >= classifier_cutoff:
    # if float(items[2]) >= 0.0:
        final_network.append(PPI)

print >> outFH, "\n".join(final_network)
outFH.close()
#############################################
## End of "MODEL TRAINING & PREDICTION"
#############################################

# Predicting clusters
utils.predict_clusters("%s.pred.txt" % (output_dir), "%s.clust.txt" % (output_dir))

('&& self.scores.shape[0]: ', 5012)
('&& pos_count: ', 244)
('&& neg_count: ', 244)
('&& self.scores.shape[0]: ', 5012)
('&& pos_count_all: ', 244)
('&& neg_count_all: ', 4768)
('&& unsure_count_all: ', 0)
('data_train: ', (488, 2, 27))
('data_train.shape: ', (488, 2, 27, 1))
('data_train: ', (5012, 2, 27))
('data_all.shape: ', (5012, 2, 27, 1))
Final Model Creation!
Processing data...
Epoch 1/200
488/488 [==============================] - 0s 47us/step - loss: 0.6938 - acc: 0.4775
Epoch 2/200
488/488 [==============================] - 0s 47us/step - loss: 0.6926 - acc: 0.5041
Epoch 3/200
488/488 [==============================] - 0s 47us/step - loss: 0.6918 - acc: 0.5102
Epoch 4/200
488/488 [==============================] - 0s 47us/step - loss: 0.6892 - acc: 0.5205
Epoch 5/200
488/488 [==============================] - 0s 46us/step - loss: 0.6855 - acc: 0.5512
Epoch 6/200
488/488 [==============================] - 0s 46us/step - loss: 0.6829 - acc: 0.5963
Epoch 7/200
488/488 [========

488/488 [==============================] - 0s 113us/step - loss: 0.2511 - acc: 0.9201
Epoch 163/200
488/488 [==============================] - 0s 113us/step - loss: 0.2358 - acc: 0.9242
Epoch 164/200
488/488 [==============================] - 0s 114us/step - loss: 0.3098 - acc: 0.8852
Epoch 165/200
488/488 [==============================] - 0s 113us/step - loss: 0.2811 - acc: 0.9098
Epoch 166/200
488/488 [==============================] - 0s 114us/step - loss: 0.2479 - acc: 0.9242
Epoch 167/200
488/488 [==============================] - 0s 114us/step - loss: 0.2906 - acc: 0.9078
Epoch 168/200
488/488 [==============================] - 0s 114us/step - loss: 0.2944 - acc: 0.8975
Epoch 169/200
488/488 [==============================] - 0s 113us/step - loss: 0.3043 - acc: 0.8934
Epoch 170/200
488/488 [==============================] - 0s 114us/step - loss: 0.2876 - acc: 0.9037
Epoch 171/200
488/488 [==============================] - 0s 124us/step - loss: 0.2314 - acc: 0.9283
Epoch 172/200


In [11]:
# Evaluating predicted clusters
pred_clusters = GS.Clusters(False)
pred_clusters.read_file("%s.clust.txt" % (output_dir))
overlapped_complexes_with_reference = gs.get_complexes().get_overlapped_complexes_set(pred_clusters)
print "# of complexes in reference dataset: " + str(len(overlapped_complexes_with_reference))
#clust_scores, header = utils.clustering_evaluation(gs.complexes, pred_clusters, "", False)
clust_scores, header, composite_score = utils.clustering_evaluation(gs.complexes, pred_clusters, "", False)
outFH = open("%s.eval.txt" % (output_dir), "w")
header = header.split("\t")
clust_scores = clust_scores.split("\t")
for i, head in enumerate(header):
    print "%s\t%s" % (head, clust_scores[i])
    print >> outFH, "%s\t%s" % (head, clust_scores[i])
outFH.close()

Average size of predicted complexes is: 476.0
# of complexes in reference dataset: 0
mmr	0.00945378151261
overlapp	0.000000
simcoe	1.000000
mean_simcoe_overlap	1.000000
sensetivity	1.0
ppv	0.0873015873015873
accuracy	0.295468420143
sep	0.188982236505
